In [ ]:
# NOISE VERSION
from PIL import Image
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torchvision.transforms.v2 as transforms

# from src.d02_binary_model import Trainer
import matplotlib.pyplot as plt

def create_images_for_counting(img_path, n, dest_path):
    any_img_path = img_path
    any_img_path += "/" + os.listdir(any_img_path)[0]
    any_img_path += "/" + os.listdir(any_img_path)[0]
    
    with open(any_img_path, "rb") as f:
        img = Image.open(f)
        img.load()

        channels = 3

        if img.mode in ("1", "L"):
            channels = 1

    max_img_amt = max(n, 4)
    
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1)
    ])
    imgs = ImageFolder(root=img_path, transform=transform if channels == 1 else None)
    
    for aspect in imgs.classes:
        indices = [i for i,(_, label) in enumerate(imgs) if label == imgs.class_to_idx[aspect]]        
        img_shape = [0] + list(imgs[indices[0]][0].size[::-1])
        
        for img_amt in range(max_img_amt + 1):
            os.makedirs(f"{dest_path}/{aspect}/h/{img_amt}")
            os.makedirs(f"{dest_path}/{aspect}/v/{img_amt}")

            h_img_num = 0
            v_img_num = 0

            for idx in indices:
                if channels == 3:
                    new_image = np.random.randint(0, 256, (img_shape[1], img_shape[2] * max_img_amt, 3), dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="RGB")
                else:
                    new_image = np.random.randint(0, 256, (img_shape[1], img_shape[2] * max_img_amt), dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="L")

                empty_space = (max_img_amt - img_amt) * img_shape[2]
                prev_position = 0

                idx_to_insert = idx
                
                for num in range(img_amt):
                    img = imgs[idx_to_insert][0]
                    shift = np.random.randint(-1, empty_space)
                    empty_space -= shift
                    new_image.paste(img, (prev_position + shift, 0))
                    prev_position += img_shape[2] + shift
                    idx_to_insert = indices[np.random.randint(len(indices))]
                
                new_image.save(f"{dest_path}/{aspect}/h/{img_amt}/{h_img_num}.jpg")
                h_img_num += 1

                if channels == 3:
                    new_image = np.random.randint(0, 256, (img_shape[1] * max_img_amt, img_shape[2], 3), dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="RGB")
                else:
                    new_image = np.random.randint(0, 256, (img_shape[1] * max_img_amt, img_shape[2]), dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="L")

                empty_space = (max_img_amt - img_amt) * img_shape[1]
                prev_position = 0

                idx_to_insert = idx

                for num in range(img_amt):
                    img = imgs[idx_to_insert][0]
                    shift = np.random.randint(-1, empty_space)
                    empty_space -= shift
                    new_image.paste(img, (0, prev_position + shift))
                    prev_position += img_shape[1] + shift
                    idx_to_insert = indices[np.random.randint(len(indices))]

                new_image.save(f"{dest_path}/{aspect}/v/{img_amt}/{v_img_num}.jpg")
                v_img_num += 1

In [ ]:
# SEGMENTS VERSION
from PIL import Image
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torchvision.transforms.v2 as transforms
from torch.utils.data import Subset
import shutil

# from src.d02_binary_model import Trainer
import matplotlib.pyplot as plt

def create_images_for_counting(img_path, n, dest_path, background_path, part):
    any_img_path = img_path
    any_img_path += "/" + os.listdir(any_img_path)[0]
    any_img_path += "/" + os.listdir(any_img_path)[0]
    
    with open(any_img_path, "rb") as f:
        img = Image.open(f)
        img.load()

        channels = 3

        if img.mode in ("1", "L"):
            channels = 1

    max_img_amt = max(n, 4)
    
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1)
    ])
    imgs = ImageFolder(root=img_path, transform=transform if channels == 1 else None)

    background_imgs = ImageFolder(root=background_path, transform=transform if channels == 1 else None)
    assembly_class_idx = background_imgs.class_to_idx["assembly"]
    part_class_idx = background_imgs.class_to_idx[part]
    valid_indices = [i for i, (_, y) in enumerate(background_imgs.samples) if y not in [assembly_class_idx, part_class_idx]] 
    background_imgs = Subset(background_imgs, valid_indices)  

    shutil.rmtree(dest_path)###########################################
    
    for aspect in imgs.classes:
        indices = [i for i,(_, label) in enumerate(imgs) if label == imgs.class_to_idx[aspect]]        
        img_shape = [0] + list(imgs[indices[0]][0].size[::-1])  # [0] not used
        
        for img_amt in range(max_img_amt + 1):
            os.makedirs(f"{dest_path}/{aspect}/h/{img_amt}")
            os.makedirs(f"{dest_path}/{aspect}/v/{img_amt}")

            h_img_num = 0
            v_img_num = 0

            for idx in indices:
                if channels == 3:
                    new_image = np.zeros([img_shape[1], img_shape[2] * img_amt, 3], dtype=np.uint8)
                    new_image = np.random.randint(0, 256, (img_shape[1], img_shape[2] * max_img_amt, 3), dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="RGB")
                else:
                    new_image = np.empty([img_shape[1], img_shape[2] * img_amt], dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="L")

                empty_space = img_shape[2] * max_img_amt
                prev_position = 0

                while empty_space > 0:
                    img_idx = np.random.randint(0, len(background_imgs))
                    left_offset = np.random.randint(0, background_imgs[img_idx][0].size[1])
                    length = np.random.randint(0, background_imgs[img_idx][0].size[0] - left_offset + 1)
                    top_offset = np.random.randint(0, background_imgs[img_idx][0].size[1] - img_shape[1])
                    height = np.random.randint(img_shape[1], background_imgs[img_idx][0].size[1] - top_offset + 1)
                    new_image.paste(background_imgs[img_idx][0].crop((left_offset, top_offset, left_offset+length, top_offset+height)), (prev_position, 0))
                    prev_position += length
                    empty_space -= length

                empty_space = (max_img_amt - img_amt) * img_shape[2]
                prev_position = 0

                idx_to_insert = idx
                
                for num in range(img_amt):
                    img = imgs[idx_to_insert][0]
                    shift = np.random.randint(-1, empty_space)
                    empty_space -= shift
                    new_image.paste(img, (prev_position + shift, 0))
                    prev_position += img_shape[2] + shift
                    idx_to_insert = indices[np.random.randint(len(indices))]                
                
                new_image.save(f"{dest_path}/{aspect}/h/{img_amt}/{h_img_num}.jpg")
                h_img_num += 1

                if channels == 3:
                    new_image = np.empty([img_shape[1] * max_img_amt, img_shape[2], 3], dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="RGB")
                else:
                    new_image = np.empty([img_shape[1] * max_img_amt, img_shape[2]], dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="L")

                empty_space = img_shape[1] * max_img_amt
                prev_position = 0

                while empty_space > 0:
                    img_idx = np.random.randint(0, len(background_imgs))
                    left_offset = np.random.randint(0, background_imgs[img_idx][0].size[0] - img_shape[2])
                    length = img_shape[2]
                    top_offset = np.random.randint(0, background_imgs[img_idx][0].size[1])
                    height = np.random.randint(0, background_imgs[img_idx][0].size[1] - top_offset + 1)
                    new_image.paste(background_imgs[img_idx][0].crop((left_offset, top_offset, left_offset+length, top_offset+height)), (0, prev_position))
                    prev_position += height
                    empty_space -= height

                empty_space = (max_img_amt - img_amt) * img_shape[1]
                prev_position = 0

                idx_to_insert = idx

                for num in range(img_amt):
                    img = imgs[idx_to_insert][0]
                    shift = np.random.randint(-1, empty_space)
                    empty_space -= shift
                    new_image.paste(img, (0, prev_position + shift))
                    prev_position += img_shape[1] + shift
                    idx_to_insert = indices[np.random.randint(len(indices))]

                new_image.save(f"{dest_path}/{aspect}/v/{img_amt}/{v_img_num}.jpg")
                v_img_num += 1

In [61]:
# FULL sIZE VERSION
from PIL import Image
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torchvision.transforms.v2 as transforms
from torch.utils.data import Subset
import shutil

# from src.d02_binary_model import Trainer
import matplotlib.pyplot as plt

def create_images_for_counting(img_path, n, dest_path, background_path, part):
    any_img_path = img_path
    any_img_path += "/" + os.listdir(any_img_path)[0]
    any_img_path += "/" + os.listdir(any_img_path)[0]
    
    with open(any_img_path, "rb") as f:
        img = Image.open(f)
        img.load()

        channels = 3

        if img.mode in ("1", "L"):
            channels = 1

    max_img_amt = max(n, 4)
    
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1)
    ])
    imgs = ImageFolder(root=img_path, transform=transform if channels == 1 else None)

    background_imgs = ImageFolder(root=background_path, transform=transform if channels == 1 else None)
    assembly_class_idx = background_imgs.class_to_idx["assembly"]
    part_class_idx = background_imgs.class_to_idx[part]
    valid_indices = [i for i, (_, y) in enumerate(background_imgs.samples) if y not in [assembly_class_idx, part_class_idx]] 
    background_imgs = Subset(background_imgs, valid_indices)  
    background_shape = [0] + list(background_imgs[0][0].size[::-1])

    shutil.rmtree(dest_path)###########################################
    
    for img_amt in range(1, max_img_amt + 1):
        os.makedirs(f"{dest_path}/{img_amt}")  
        img_num = 0

        for aspect in imgs.classes:
            indices = [i for i,(_, label) in enumerate(imgs) if label == imgs.class_to_idx[aspect]]     
            img_shape = [0] + list(imgs[indices[0]][0].size[::-1])  # [0] not used
            
            for idx in indices:  
                if channels == 3:
                    new_image = np.zeros((*background_shape[1:3], 3), dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="RGB")
                else:
                    new_image = np.zeros( (*background_shape[1:2],), dtype=np.uint8)
                    new_image = Image.fromarray(new_image, mode="L")

                background_img_idx = np.random.randint(len(background_imgs))

                position = 0
                idx_to_insert = idx
                random_imgs_num = (background_shape[1] // img_shape[1]) * (background_shape[2] // img_shape[2]) - img_amt

                for num in range(max_img_amt):
                    if random_imgs_num > 0 and (np.random.randint(2) == 1 or num + random_imgs_num == max_img_amt):
                        background_img_idx = np.random.randint(len(background_imgs))
                        left_offset = np.random.randint(0, background_shape[2] - img_shape[2])
                        top_offset = np.random.randint(0, background_shape[1] - img_shape[1])
                        img = background_imgs[background_img_idx][0].crop((left_offset, top_offset, left_offset+img_shape[2], top_offset+img_shape[1]))
                        
                        random_imgs_num -= 1
                    else:
                        img = imgs[idx_to_insert][0]
                        idx_to_insert = indices[np.random.randint(len(indices))]  

                    new_image.paste(img, (position, num // (background_shape[2] // img_shape[2]) * img_shape[1]))

                    position += img_shape[2]
                    position %= background_shape[2]              
                
                new_image.save(f"{dest_path}/{img_amt}/{img_num}_{img_amt}.jpg")
                img_num += 1

In [62]:
import os

folder = "D:/layer_imgs_mobilenet_v1/4/"
background_folder = "./data/temp"

for ass in ["wheels_five"]:#os.listdir(folder):
    for scale in ["scale_0.2"]:#os.listdir(f"{folder}/{ass}/"):
        for part in ["screw"]:#os.listdir(f"{folder}/{ass}/{scale}"):
            print(f"{ass}/{scale}/{part}")
            create_images_for_counting(f"{folder}/{ass}/{scale}/{part}", 6, f"D:/counting_imgs_test/{ass}/{scale}/{part}", f"{background_folder}/{ass}/{scale}", part)

wheels_five/scale_0.2/screw


In [5]:
from torchvision.models import mobilenet_v3_large, MobileNetV2
from torchvision.models import squeezenet1_1, shufflenet_v2_x1_5
from torchvision.models.detection import ssdlite320_mobilenet_v3_large

l = squeezenet1_1(weights="SqueezeNet1_1_Weights.IMAGENET1K_V1")
#l = shufflenet_v2_x1_5(weights="ShuffleNet_V2_X1_5_Weights.IMAGENET1K_V1")
print(l)   

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [7]:
path = "./data/temp/counting_imgs/wheels_five/scale_0.2/screw/3_2/h"
part='screw'
path = path[:path.rfind(part)+len(part)]
model_name = path.replace("/", "_")
#._data_temp_counting_imgs_wheels_five_scale_0.2_screw_counter
model_name

'._data_temp_counting_imgs_wheels_five_scale_0.2_screw'